In [2]:
# General libraries
import geopandas as gpd
import gymnasium as gym
import importlib
import numpy as np

# RL libraries
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.utils import get_action_masks
from stable_baselines3.common.env_util import make_vec_env

# MAAT environment
import maat.world as world

# Filter out the FutureWarning
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import logging
logging.getLogger('climada').setLevel(logging.ERROR)  # to silence all warnings

# Constants
DATA_LOCATION_LOCAL = "data" # data in this folder does not reflect the original data used, but its rather just used for demonstration purposes
DATA_LOCATION_O_DRIVE = "data" # data in this folder does not reflect the original data used, but its rather just used for demonstration purposes

CRS_25832 = 'epsg:25832'
CRS_4326 = 'epsg:4326'

In [ ]:
importlib.reload(world)

env_id = 'maat/BasicEnvironment-v1' # It is best practice to have a space name and version number.

gym.envs.registration.register(
    id=env_id,
    entry_point=world.BasicEnvironment,
    max_episode_steps=100,
    reward_threshold=0 
)
env = make_vec_env(
    env_id, 
    n_envs=1, 
    seed=0, 
    env_kwargs={
        "assets_types": ["network"],
        "render_mode": "ansi",
        "hazard_sampling_scheme": world.HazardSampling.DETERMINISTIC,
        "preload_hazards": True,
        "hazards_to_preload": [160],
        "city_zones": "IndreBy",
        "episode_time_steps": 77,
        })

/home/mncosta/anaconda3/envs/climada39/lib/python3.9/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment maat/BasicEnvironment-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


ipfn converged: convergence_rate not updating or below rate_tolerance


In [10]:
env.step(1)

TypeError: 'int' object is not subscriptable

In [ ]:
model = MaskablePPO("MlpPolicy", 
                    env, 
                    learning_rate=0.01,
                    n_steps=48,
                    batch_size=48,
                    n_epochs=4,
                    ent_coef=0.1,
                    tensorboard_log="./tensorboard/",
                    verbose=1)

Using cuda device


In [ ]:
model.learn(total_timesteps=8000,
            log_interval=3, 
            progress_bar=False, 
            tb_log_name="MaskablePPO_77ts_IndreBy_", 
            # callback=[checkpoint_callback, eval_callback],
            )

Logging to ./dqn_world_tensorboard/MaskablePPO_40ts_IndreBy__2
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 40         |
|    ep_rew_mean          | 25.5       |
| time/                   |            |
|    fps                  | 0          |
|    iterations           | 3          |
|    time_elapsed         | 267        |
|    total_timesteps      | 144        |
| train/                  |            |
|    approx_kl            | 0.08031764 |
|    clip_fraction        | 0.474      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.23      |
|    explained_variance   | 0.0289     |
|    learning_rate        | 0.01       |
|    loss                 | 15.3       |
|    n_updates            | 8          |
|    policy_gradient_loss | -0.052     |
|    value_loss           | 37.1       |
----------------------------------------
----------------------------------------
| rollout/                |        

In [ ]:
model.save("models/MaskablePPO_77ts_IndreBy_withrain_3000")

In [110]:
model = MaskablePPO.load("models/MaskablePPO_77ts_IndreBy_withrain_3000")

In [ ]:
importlib.reload(world)

# instantiate the environment
env = world.BasicEnvironment(assets_types=["network"],
                             render_mode="ansi",
                             hazard_sampling_scheme=world.HazardSampling.DETERMINISTIC,
                             preload_hazards=True,
                             hazards_to_preload=[160],
                             city_zones="IndreBy",
                             episode_time_steps=77,)

In [98]:
cost_reward_epi = []
cost_action_epi = []
cost_impacts_epi = []
time_diff_epi= []
hours_delays_epi = []

for _ in range(10):

    obs, _ = env.reset()

    cost_action = []
    cost_impacts = []
    time_diff = []
    hours_delays = []

    # for _ in range(env.max_time_steps):
    for _ in range(2035-2023):
        # Retrieve current action mask
        action_masks = get_action_masks(env)

        action, _states = model.predict(obs, action_masks=action_masks)
        obs, reward, terminated, truncated, info = env.step(action)
        # print(action, reward, info["cumulative_reward"], terminated,)

        cost_impacts.append(info["impacts_direct_damage"].sum() + info["impacts_travel_delays"].sum())
        cost_action.append(info["impacts_action_cost"].sum())
        hours_delays.append(info["impacts_travel_delays"].sum() / 213)
        time_diff.append(np.array([(info["transport_network"].edges[edge]["travel_time_impacted"]-info["transport_network"].edges[edge]["travel_time"]) for edge in info["transport_network"].edges]).mean())
        
        print("{:.2f}M".format(np.array(info["reward_monetary_over_time"]).sum()/1e6), "\t",
              "{:.2f}M".format(np.array(cost_action).sum()/1e6), "\t",
              "{:.2f}M".format(np.array(cost_impacts).sum()/1e6), "\t",
              "{:.2f}h".format(np.array(time_diff).mean()), "\t",
              "{:.2f}h".format(np.array(hours_delays).sum()/24)
              )
        
    cost_reward_epi.append(np.array(info["reward_monetary_over_time"]).sum())
    cost_action_epi.append(np.array(cost_action).sum())
    cost_impacts_epi.append(np.array(cost_impacts).sum())
    hours_delays_epi.append(np.array(hours_delays).sum()/24)

cost_reward_epi = np.array(cost_reward_epi)
cost_action_epi = np.array(cost_action_epi)
cost_impacts_epi = np.array(cost_impacts_epi)
hours_delays_epi = np.array(hours_delays_epi)

print("Final results:")
print("{:.2f}".format(cost_reward_epi.mean() /1e6), "+/-", "{:.2f}".format(cost_reward_epi.std() /1e6), "\n",
      "{:.2f}".format(cost_action_epi.mean() /1e6), "+/-", "{:.2f}".format(cost_action_epi.std() /1e6), "\n",
      "{:.2f}".format(cost_impacts_epi.mean()/1e6), "+/-", "{:.2f}".format(cost_impacts_epi.std()/1e6), "\n",
      "{:.2f}".format(hours_delays_epi.mean()/1e3), "+/-", "{:.2f}".format(hours_delays_epi.std()/1e3)
      )

-3.27M 	 0.45M 	 2.82M 	 1.33h 	 498.49h
-7.54M 	 3.60M 	 3.94M 	 1.22h 	 669.98h
-10.91M 	 5.85M 	 5.06M 	 1.19h 	 841.46h
-13.56M 	 7.38M 	 6.18M 	 1.17h 	 1012.94h
-15.21M 	 7.91M 	 7.30M 	 1.16h 	 1184.42h
-18.50M 	 10.10M 	 8.40M 	 1.15h 	 1355.91h
-20.05M 	 10.55M 	 9.50M 	 1.15h 	 1527.39h
-24.80M 	 14.23M 	 10.57M 	 1.14h 	 1698.87h
-26.44M 	 14.81M 	 11.63M 	 1.14h 	 1870.36h
-28.03M 	 15.33M 	 12.70M 	 1.14h 	 2041.84h
-30.91M 	 17.54M 	 13.37M 	 1.07h 	 2137.44h
-35.52M 	 21.51M 	 14.02M 	 1.01h 	 2233.04h
-3.27M 	 0.45M 	 2.82M 	 1.33h 	 498.49h
-7.54M 	 3.60M 	 3.94M 	 1.22h 	 669.98h
-10.91M 	 5.85M 	 5.06M 	 1.19h 	 841.46h
-13.56M 	 7.38M 	 6.18M 	 1.17h 	 1012.94h
-16.49M 	 9.59M 	 6.90M 	 1.01h 	 1108.54h
-19.38M 	 11.77M 	 7.61M 	 0.90h 	 1204.15h
-20.62M 	 12.31M 	 8.32M 	 0.82h 	 1299.75h
-21.91M 	 12.89M 	 9.02M 	 0.76h 	 1395.35h
-23.06M 	 13.34M 	 9.72M 	 0.72h 	 1490.95h
-27.99M 	 17.58M 	 10.41M 	 0.68h 	 1586.55h
-32.62M 	 21.55M 	 11.07M 	 0.65h 	 1682.16h
-

In [111]:
import folium
import branca.colormap as cmp
from IPython.core.display import display, HTML
import binascii
from io import BytesIO
import matplotlib


obs, _ = env.reset()

for _ in range(2035-2023):
    # Retrieve current action mask
    action_masks = get_action_masks(env)

    action, _states = model.predict(obs, action_masks=action_masks)
    obs, reward, terminated, truncated, info = env.step(action)
    # print(action, reward, info["cumulative_reward"], terminated,)


# ======================================================== #
# Map 1
# ======================================================== #
map_render1 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
colormap1 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=env.taz.impacts_direct_damage.max()) 
colormap1.caption = "Road Damage (DKK)"
map_render1.add_child(colormap1)

for _, r in env.taz.to_crs(CRS_4326).iterrows():
    folium.GeoJson(r["geometry"],
                   fillColor=colormap1(r["impacts_direct_damage"]),
                   fillOpacity=0.7,
                   color="black",
                   weight=.3,
                   ).add_to(map_render1)

# Add base layers and controls
folium.TileLayer('cartodbpositron').add_to(map_render1)

# ======================================================== #
# Map 2
# ======================================================== #
map_render2 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
colormap2 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=env.taz.impacts_travel_delays.max()) 
colormap2.caption = "Travel Delays (DKK)"
map_render2.add_child(colormap2)

for _, r in env.taz.to_crs(CRS_4326).iterrows():
    folium.GeoJson(r["geometry"],
                   fillColor=colormap2(r["impacts_travel_delays"]),
                   fillOpacity=0.7,
                   color="black",
                   weight=.3,
                   ).add_to(map_render2)

# Add base layers and controls
folium.TileLayer('cartodbpositron').add_to(map_render2)

# ======================================================== #
# Map 3
# ======================================================== #
map_render3 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
folium.TileLayer('cartodbpositron').add_to(map_render3)
network_layer = folium.FeatureGroup(name="Network", 
                                            legend_name="Transport Network",
                                            show=True)

colormap3 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=100)
colormap3.caption = "Travel Time Delays (%)"
map_render3.add_child(colormap3)

# Draw TAZ
for node in env.transport_network.nodes:
    folium.GeoJson(
        env.taz[env.taz['zoneid'] == node]['geometry'],
        opacity=.2,
        fill_opacity=.1,
        color='black',
        fill_color='black',
        weight=1,
        ).add_to(network_layer)

# Draw edges
for edge in env.transport_network.edges:
    perc_diff = 100*(env.transport_network.edges[edge]['travel_time_impacted'] - env.transport_network.edges[edge]['travel_time']) / env.transport_network.edges[edge]['travel_time']
    
    # Draw the edge
    folium.PolyLine(
        locations=[env.transport_network.nodes[edge[0]]['centroid_4326'][::-1],
                   env.transport_network.nodes[edge[1]]['centroid_4326'][::-1]],
        #color='black',
        weight=5,
        popup="Original: {:.2f}s<br>Impacted: {:.2f}s".format(env.transport_network.edges[edge]['travel_time'],
                                        env.transport_network.edges[edge]['travel_time_impacted']),
        color=colormap3(perc_diff),
    ).add_to(network_layer)

# Draw nodes
for node in env.transport_network.nodes:
    folium.CircleMarker(
        location=env.transport_network.nodes[node]['centroid_4326'][::-1],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.,
        popup="{}<br>{}".format(env.taz[env.taz['zoneid'] == node]['zoneid'].values,
                                env.taz[env.taz['zoneid'] == node]['zonenavn'].values[0]),
    ).add_to(network_layer)

network_layer.add_to(map_render3);

# ======================================================== #
# Map 4
# ======================================================== #
map_render4 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
folium.TileLayer('cartodbpositron').add_to(map_render4)

for _, r in env.taz.to_crs(CRS_4326).iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "lightblue",
                                                     "color": "lightblue"})
    folium.Popup(r["zonenavn"]).add_to(geo_j)
    geo_j.add_to(map_render4)

for i, action_i in enumerate(env.previous_actions):
    zone_id = env._zone_ids[action_i-1]
    zone = env.taz[env.taz["zoneid"] == zone_id].to_crs(CRS_4326)
    sim_geo = gpd.GeoSeries(zone["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "red",
                                                     "color": "red"})
    folium.Popup(zone["zonenavn"].values[0]).add_to(geo_j)
    geo_j.add_to(map_render4)

# ======================================================== #
# Draw maps
# ======================================================== #
htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               .format(map_render1.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render2.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render3.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render4.get_root().render().replace('"', '&quot;'), 600, 450))
display(htmlmap)

/tmp/ipykernel_125738/2809425545.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/home/mncosta/maat_drl/.conda/lib/python3.9/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [12]:
import folium
from folium.features import DivIcon

map_render = folium.Map(location=[55.68539883157378, 12.58535859765494], zoom_start=14)

for _, r in env.taz.to_crs(CRS_4326).iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "lightblue",
                                                     "color": "lightblue"})
    folium.Popup(r["zonenavn"]).add_to(geo_j)
    geo_j.add_to(map_render)

for i, action_i in enumerate(env.previous_actions):
    if action_i == 0:
        continue
    zone_id = env._zone_ids[action_i-1]
    zone = env.taz[env.taz["zoneid"] == zone_id].to_crs(CRS_4326)
    sim_geo = gpd.GeoSeries(zone["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "red",
                                                     "color": "red"})
    folium.Popup(zone["zonenavn"].values[0]).add_to(geo_j)
    geo_j.add_to(map_render)
    

    folium.Marker(zone["centroid_4326"].values[0][::-1],
                  icon=DivIcon(
                    icon_size=(100,36),
                    icon_anchor=(0,0),
                    html='<div style="font-size: 16pt">{}</div>'.format(i+1),
                    ),
                  popup="{}".format(i+1)).add_to(map_render)



folium.TileLayer('cartodbpositron').add_to(map_render)
map_render

In [ ]:
env._render_human()

## Random Agent

In [ ]:
importlib.reload(world)

# instantiate the environment
env = world.BasicEnvironment(assets_types=["network"],
                             render_mode="ansi",
                             hazard_sampling_scheme=world.HazardSampling.DETERMINISTIC,
                             preload_hazards=True,
                             hazards_to_preload=[160],
                             city_zones="IndreBy",
                             episode_time_steps=77,)

In [100]:
cost_reward_epi = []
cost_action_epi = []
cost_impacts_epi = []
time_diff_epi= []
hours_delays_epi = []

for _ in range(10):

    obs, _ = eval_env.reset()

    cost_action = []
    cost_impacts = []
    time_diff = []
    hours_delays = []

    # for _ in range(eval_env.max_time_steps):
    for _ in range(2035-2023):
        # Retrieve current action mask
        action_masks = get_action_masks(eval_env)

        action, _states = model_random.predict(obs, action_masks=action_masks)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        # print(action, reward, info["cumulative_reward"], terminated,)

        cost_impacts.append(info["impacts_direct_damage"].sum() + info["impacts_travel_delays"].sum())
        cost_action.append(info["impacts_action_cost"].sum())
        hours_delays.append(info["impacts_travel_delays"].sum() / 213)
        time_diff.append(np.array([(info["transport_network"].edges[edge]["travel_time_impacted"]-info["transport_network"].edges[edge]["travel_time"]) for edge in info["transport_network"].edges]).mean())
        
        print("{:.2f}M".format(np.array(info["reward_monetary_over_time"]).sum()/1e6), "\t",
              "{:.2f}M".format(np.array(cost_action).sum()/1e6), "\t",
              "{:.2f}M".format(np.array(cost_impacts).sum()/1e6), "\t",
              "{:.2f}h".format(np.array(time_diff).mean()), "\t",
              "{:.2f}h".format(np.array(hours_delays).sum()/24)
              )
        
    cost_reward_epi.append(np.array(info["reward_monetary_over_time"]).sum())
    cost_action_epi.append(np.array(cost_action).sum())
    cost_impacts_epi.append(np.array(cost_impacts).sum())
    hours_delays_epi.append(np.array(hours_delays).sum()/24)

cost_reward_epi = np.array(cost_reward_epi)
cost_action_epi = np.array(cost_action_epi)
cost_impacts_epi = np.array(cost_impacts_epi)
hours_delays_epi = np.array(hours_delays_epi)

print("Final results:")
print("{:.2f}".format(cost_reward_epi.mean() /1e6), "+/-", "{:.2f}".format(cost_reward_epi.std() /1e6), "\n",
      "{:.2f}".format(cost_action_epi.mean() /1e6), "+/-", "{:.2f}".format(cost_action_epi.std() /1e6), "\n",
      "{:.2f}".format(cost_impacts_epi.mean()/1e6), "+/-", "{:.2f}".format(cost_impacts_epi.std()/1e6), "\n",
      "{:.2f}".format(hours_delays_epi.mean()/1e3), "+/-", "{:.2f}".format(hours_delays_epi.std()/1e3)
      )

-3.58M 	 0.76M 	 2.82M 	 1.33h 	 498.49h
-7.20M 	 1.56M 	 5.64M 	 1.33h 	 996.99h
-13.96M 	 5.52M 	 8.44M 	 1.33h 	 1495.48h
-21.43M 	 10.20M 	 11.23M 	 1.33h 	 1993.98h
-25.38M 	 11.36M 	 14.02M 	 1.33h 	 2492.47h
-29.28M 	 12.48M 	 16.81M 	 1.33h 	 2990.96h
-32.52M 	 12.93M 	 19.59M 	 1.33h 	 3489.46h
-37.15M 	 14.78M 	 22.37M 	 1.33h 	 3987.95h
-41.78M 	 16.63M 	 25.15M 	 1.33h 	 4486.44h
-46.37M 	 18.84M 	 27.53M 	 1.25h 	 4909.06h
-51.58M 	 21.73M 	 29.84M 	 1.19h 	 5331.67h
-54.34M 	 22.18M 	 32.16M 	 1.14h 	 5754.28h
-4.64M 	 2.21M 	 2.42M 	 0.57h 	 422.61h
-8.91M 	 4.06M 	 4.84M 	 0.57h 	 845.23h
-12.45M 	 5.18M 	 7.26M 	 0.57h 	 1267.84h
-17.04M 	 7.37M 	 9.67M 	 0.57h 	 1690.45h
-19.96M 	 7.89M 	 12.07M 	 0.57h 	 2113.07h
-22.81M 	 8.34M 	 14.47M 	 0.57h 	 2535.68h
-26.01M 	 9.14M 	 16.87M 	 0.57h 	 2958.29h
-33.09M 	 13.82M 	 19.27M 	 0.57h 	 3380.90h
-39.14M 	 17.51M 	 21.63M 	 0.57h 	 3803.52h
-43.75M 	 19.75M 	 23.99M 	 0.57h 	 4226.13h
-47.26M 	 20.92M 	 26.35M 	 0.57h 	

In [108]:
import folium
import branca.colormap as cmp
from IPython.core.display import display, HTML
import binascii
from io import BytesIO
import matplotlib


obs, _ = eval_env.reset()

for _ in range(2035-2023):
    # Retrieve current action mask
    action_masks = get_action_masks(eval_env)

    action, _states = model_random.predict(obs, action_masks=action_masks)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    # print(action, reward, info["cumulative_reward"], terminated,)


# ======================================================== #
# Map 1
# ======================================================== #
map_render1 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
colormap1 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=eval_env.taz.impacts_direct_damage.max()) 
colormap1.caption = "Road Damage (DKK)"
map_render1.add_child(colormap1)

for _, r in eval_env.taz.to_crs(CRS_4326).iterrows():
    folium.GeoJson(r["geometry"],
                   fillColor=colormap1(r["impacts_direct_damage"]),
                   fillOpacity=0.7,
                   color="black",
                   weight=.3,
                   ).add_to(map_render1)

# Add base layers and controls
folium.TileLayer('cartodbpositron').add_to(map_render1)

# ======================================================== #
# Map 2
# ======================================================== #
map_render2 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
colormap2 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=eval_env.taz.impacts_travel_delays.max()) 
colormap2.caption = "Travel Delays (DKK)"
map_render2.add_child(colormap2)

for _, r in eval_env.taz.to_crs(CRS_4326).iterrows():
    folium.GeoJson(r["geometry"],
                   fillColor=colormap2(r["impacts_travel_delays"]),
                   fillOpacity=0.7,
                   color="black",
                   weight=.3,
                   ).add_to(map_render2)

# Add base layers and controls
folium.TileLayer('cartodbpositron').add_to(map_render2)

# ======================================================== #
# Map 3
# ======================================================== #
map_render3 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
folium.TileLayer('cartodbpositron').add_to(map_render3)
network_layer = folium.FeatureGroup(name="Network", 
                                            legend_name="Transport Network",
                                            show=True)

colormap3 = cmp.LinearColormap(["white", "yellow", "orange", "red"], vmin=0, vmax=100)
colormap3.caption = "Travel Time Delays (%)"
map_render3.add_child(colormap3)

# Draw TAZ
for node in eval_env.transport_network.nodes:
    folium.GeoJson(
        eval_env.taz[eval_env.taz['zoneid'] == node]['geometry'],
        opacity=.2,
        fill_opacity=.1,
        color='black',
        fill_color='black',
        weight=1,
        ).add_to(network_layer)

# Draw edges
for edge in eval_env.transport_network.edges:
    perc_diff = 100*(eval_env.transport_network.edges[edge]['travel_time_impacted'] - eval_env.transport_network.edges[edge]['travel_time']) / eval_env.transport_network.edges[edge]['travel_time']
    
    # Draw the edge
    folium.PolyLine(
        locations=[eval_env.transport_network.nodes[edge[0]]['centroid_4326'][::-1],
                   eval_env.transport_network.nodes[edge[1]]['centroid_4326'][::-1]],
        #color='black',
        weight=5,
        popup="Original: {:.2f}s<br>Impacted: {:.2f}s".format(eval_env.transport_network.edges[edge]['travel_time'],
                                        eval_env.transport_network.edges[edge]['travel_time_impacted']),
        color=colormap3(perc_diff),
    ).add_to(network_layer)

# Draw nodes
for node in eval_env.transport_network.nodes:
    folium.CircleMarker(
        location=eval_env.transport_network.nodes[node]['centroid_4326'][::-1],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.,
        popup="{}<br>{}".format(eval_env.taz[eval_env.taz['zoneid'] == node]['zoneid'].values,
                                eval_env.taz[eval_env.taz['zoneid'] == node]['zonenavn'].values[0]),
    ).add_to(network_layer)

network_layer.add_to(map_render3);

# ======================================================== #
# Map 4
# ======================================================== #
map_render4 = folium.Map(location=[55.68539883157378, 12.58535859765494], 
                        zoom_start=13,
                        width=600, height=400,
                        zoom_control=False,
                        scrollWheelZoom=False,
                        dragging=False)
folium.TileLayer('cartodbpositron').add_to(map_render4)

for _, r in eval_env.taz.to_crs(CRS_4326).iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "lightblue",
                                                     "color": "lightblue"})
    folium.Popup(r["zonenavn"]).add_to(geo_j)
    geo_j.add_to(map_render4)

for i, action_i in enumerate(eval_env.previous_actions):
    zone_id = eval_env._zone_ids[action_i-1]
    zone = eval_env.taz[eval_env.taz["zoneid"] == zone_id].to_crs(CRS_4326)
    sim_geo = gpd.GeoSeries(zone["geometry"])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function=lambda x: {"fillColor": "red",
                                                     "color": "red"})
    folium.Popup(zone["zonenavn"].values[0]).add_to(geo_j)
    geo_j.add_to(map_render4)

# ======================================================== #
# Draw maps
# ======================================================== #
htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 48%; margin: 0 auto; border: 0px solid black"></iframe>'
               .format(map_render1.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render2.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render3.get_root().render().replace('"', '&quot;'), 600, 450,
                       map_render4.get_root().render().replace('"', '&quot;'), 600, 450))
display(htmlmap)

/tmp/ipykernel_125738/2514354488.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/home/mncosta/maat_drl/.conda/lib/python3.9/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
